In [0]:
storage_account_name = "strgacc2831"
storage_account_key = "nK93TB+8Q47M4LiV45af/QN9W01RMS1NJYTe3ITBfE/QetSi7jp2jAtmC/DsNenMGt5aNqGjDBxR+AStA8o5SA=="

# Configure Spark to use the storage account key
spark.conf.set(f"fs.azure.account.key.strgacc2831.dfs.core.windows.net","nK93TB+8Q47M4LiV45af/QN9W01RMS1NJYTe3ITBfE/QetSi7jp2jAtmC/DsNenMGt5aNqGjDBxR+AStA8o5SA==")

In [0]:
# Assuming both files are in CSV format
accounts_df = spark.read.format("csv").option("header", "true").load("abfss://bronze@strgacc2831.dfs.core.windows.net/accounts.csv")
customers_df = spark.read.format("csv").option("header", "true").load("abfss://bronze@strgacc2831.dfs.core.windows.net/customers.csv")
loan_payments_df = spark.read.format("csv").option("header", "true").load("abfss://bronze@strgacc2831.dfs.core.windows.net/loan_payments.csv")
loans_df = spark.read.format("csv").option("header", "true").load("abfss://bronze@strgacc2831.dfs.core.windows.net/loans.csv")
transactions_df = spark.read.format("csv").option("header", "true").load("abfss://bronze@strgacc2831.dfs.core.windows.net/transactions.csv")

accounts_df.show(5)
customers_df.show(5)
loan_payments_df.show(5)
loans_df.show(5)
transactions_df.show(5)

+----------+-----------+------------+-------+
|account_id|customer_id|account_type|balance|
+----------+-----------+------------+-------+
|         1|         45|     Savings|1000.50|
|         2|         12|    Checking|2500.75|
|         3|         78|     Savings|1500.00|
|         4|         34|    Checking|3000.25|
|         5|         56|     Savings| 500.00|
+----------+-----------+------------+-------+
only showing top 5 rows

+-----------+----------+---------+--------------+---------+-----+------+
|customer_id|first_name|last_name|       address|     city|state|   zip|
+-----------+----------+---------+--------------+---------+-----+------+
|          1|      John|      Doe|    123 Elm St|  Toronto|   ON|M4B1B3|
|          2|      Jane|    Smith| 456 Maple Ave|   Ottawa|   ON|K1A0B1|
|          3|   Michael|  Johnson|    789 Oak Dr| Montreal|   QC|H1A1A1|
|          4|     Emily|    Davis|   101 Pine Rd|  Calgary|   AB|T2A0A1|
|          5|     David|   Wilson|202 Birch Blvd|V

In [0]:
# Select only the customer_id column from the customer DataFrame
valid_customers = customers_df.select("customer_id").distinct()

# Join accounts with valid_customers on customer_id
cleaned_accounts_df = accounts_df.join(valid_customers, on="customer_id", how="inner")

cleaned_accounts_df.show(20)

+-----------+----------+------------+-------+
|customer_id|account_id|account_type|balance|
+-----------+----------+------------+-------+
|          7|        28|    Checking|2900.00|
|         51|        41|     Savings| 250.25|
|         15|        38|    Checking|3900.50|
|         54|        89|     Savings| 850.25|
|         11|        24|    Checking|2600.00|
|         29|        13|     Savings|1300.25|
|         69|        65|     Savings| 550.25|
|         42|        54|    Checking|5500.50|
|         73|        71|     Savings| 625.75|
|         87|        73|     Savings| 650.25|
|         64|        14|    Checking|3200.50|
|          3|        11|     Savings|1100.75|
|         30|        80|    Checking|8100.00|
|         34|         4|    Checking|3000.25|
|         59|        69|     Savings| 600.25|
|          8|        68|    Checking|6900.00|
|         22|        76|    Checking|7700.00|
|         28|        56|    Checking|5700.00|
|         85|        33|     Savin

In [0]:
cleaned_accounts_df.write.format("csv").mode("overwrite").option("header", "true").save("abfss://silver@strgacc2831.dfs.core.windows.net/cleaned_accounts.csv")
accounts_df.show(10)

+----------+-----------+------------+-------+
|account_id|customer_id|account_type|balance|
+----------+-----------+------------+-------+
|         1|         45|     Savings|1000.50|
|         2|         12|    Checking|2500.75|
|         3|         78|     Savings|1500.00|
|         4|         34|    Checking|3000.25|
|         5|         56|     Savings| 500.00|
|         6|         23|    Checking|1200.50|
|         7|         89|     Savings| 800.75|
|         8|         67|    Checking|2200.00|
|         9|         14|     Savings| 900.25|
|        10|         92|    Checking|1800.50|
+----------+-----------+------------+-------+
only showing top 10 rows



In [0]:
# Remove rows with null values in important columns
accounts_df = accounts_df.dropna(subset=["account_id", "balance"])

# Remove duplicate rows
accounts_df = accounts_df.dropDuplicates(["account_id"])
accounts_df.show(10)


+----------+-----------+------------+-------+
|account_id|customer_id|account_type|balance|
+----------+-----------+------------+-------+
|         7|         89|     Savings| 800.75|
|        51|         72|     Savings| 375.75|
|        15|         47|     Savings| 700.75|
|        54|         42|    Checking|5500.50|
|        11|          3|     Savings|1100.75|
|        29|         58|     Savings|  75.25|
|        69|         59|     Savings| 600.25|
|        42|         36|    Checking|4300.50|
|        73|         87|     Savings| 650.25|
|        87|         93|     Savings| 825.75|
+----------+-----------+------------+-------+
only showing top 10 rows



In [0]:
# Filter out accounts with negative balances if not required
accounts_df = accounts_df.filter(accounts_df.balance >= 0)
accounts_df.show(10)

+----------+-----------+------------+-------+
|account_id|customer_id|account_type|balance|
+----------+-----------+------------+-------+
|         7|         89|     Savings| 800.75|
|        51|         72|     Savings| 375.75|
|        15|         47|     Savings| 700.75|
|        54|         42|    Checking|5500.50|
|        11|          3|     Savings|1100.75|
|        29|         58|     Savings|  75.25|
|        69|         59|     Savings| 600.25|
|        42|         36|    Checking|4300.50|
|        73|         87|     Savings| 650.25|
|        87|         93|     Savings| 825.75|
+----------+-----------+------------+-------+
only showing top 10 rows



In [0]:
# Save cleaned data as Delta format for efficient storage
accounts_df.write.format("delta").mode("overwrite").save("/mnt/datalake/silver/accounts_cleaned")
customers_df.write.format("delta").mode("overwrite").save("/mnt/datalake/silver/customers_cleaned")